In [1]:
# App that just sends information to the data preprocessor (proc_data)

In [29]:
import numpy as np
import pandas as pd
import os
import json
import requests
import obspy

from flask import Flask, request, jsonify

import tkinter as tk
from tkinter import filedialog

from sqlalchemy import create_engine

In [30]:
# Import configuration paramaters for pipeline
import pconf

In [31]:
# App vriables
app = Flask(__name__)
app_name = 'starter'
napp = 'proc_data'
assoc_feed = 'assoc_feed'
model_par = {v[0]:v[1] for v in [pair.split(':') for pair in pconf.model_name[:-3].split('|')]}

In [32]:
# Tk settings
root = tk.Tk()
root.withdraw()

''

In [33]:
# Create a SQLAlchemy connection to the site table
site_con = create_engine('sqlite:///site.db', echo=False)

In [34]:
pd.read_sql_table('site', site_con)

,index,STA,ONDATE,OFFDATE,LAT,LON,ELEV
0,0,ZAA0,2006286,2286324,53.948063,84.818807,0.2294
1,1,ZAA0B,2006286,2286324,53.948051,84.818784,0.2294
2,2,ZAA1,2006286,2286324,53.952416,84.819022,0.2462
3,3,ZAA2,2006286,2286324,53.943938,84.823645,0.2071
4,4,ZAA3,2006286,2286324,53.947681,84.809064,0.2428
...,...,...,...,...,...,...,...
2213,2213,ERM07,2004271,2286324,44.045240,110.882645,1.1707
2214,2214,ERM08,2004271,2286324,44.043966,110.858792,1.1777
2215,2215,ERM09,2004271,2286324,44.063787,110.843534,1.1838
2216,2216,ERM31,2004271,2286324,44.060498,110.872191,1.1701


In [39]:
# Read table into pandas dataframe
site_df = pd.read_sql_table('site', site_con).set_index('index')
# Remove the name of the index
site_df.index.name = None

In [ ]:
# Request dictionary
req = {}

files = filedialog.askopenfilenames()

req['window_size'] = 60 * 60
req['overlap'] = 45 * 60
req['n_windows'] = 2  # 0 processes all the data until the file ends
req['f_low'] = float(model_par['f_low'])
req['f_hig'] = float(model_par['f_hig'])
req['r_smp'] = int(model_par['r_smp'])
req['dt'] = 0.2
req['c_len'] = int(model_par['c_len'])
req['cat'] = 'raw_catalog.csv'
req['clean_cat'] = 'clean_catalog.csv'
req['events'] = 'events.csv'
req['files'] = []

# Number of seconds not to trust the picks in front or end of the characteristic function
req['trim'] = int(model_par['left_trim'])


for f in files:
    # Filename must be:
    # .mseed extension
    # STA_DATE.mseed
    sta, file = f.split('/')[-1].split('_')
    try:
        lat = site_df[site_df.STA == sta].LAT.values[0]
        lon = site_df[site_df.STA == sta].LON.values[0]
    except:
        site_df = pd.read_pickle("TA_Catalog.zip")
        lat = site_df[site_df.STA == sta].LAT.values[0]
        lon = site_df[site_df.STA == sta].LON.values[0]
        pass
    try:
        lat
    except:
        raise

    metadata = {
        'data_file': f,
        'sta': sta,
        'st_lat': lat,
        'st_lon': lon
    }
    req['files'].append(metadata)

In [37]:
# Send the post request and store the returned value in the response object
picker_response = requests.post(f'http://{pconf.host}:{pconf.apps[napp]}/{napp}', json = req, headers = pconf.head)

In [38]:
# Send trigger to associator feed to clean the catalog
pre_assoc_response = requests.post(f'http://{pconf.host}:{pconf.apps[assoc_feed]}/{assoc_feed}', json = picker_response.json(), headers=pconf.head)